In [ ]:
from notebook_prelude import *

## Create condor scheduling jobs

In [ ]:
from utils import time_utils

In [ ]:
all_experiments = glob('{}/*/*.yaml'.format(EXPERIMENT_CONFIG_FOLDER))

priorities = dict(
    split_multi_words=3,
    edge_labels=2,
    relabel=1,
    remove_infrequent=1,
    content_vs_structure=1,
    combined=0,
    use_directed=0,
    node_weights=-1,
    dimensionality_reduction=-999,
    ngrams=-2
)

verbose = 1
cores = 16
extra = '--use_nested'
extra = ''
create_predictions='true'

tmpl = 'condor_submit priority="{prio}" batch_name="{name}" Args="--n_jobs {cores} --verbose {verbose} --create_predictions {create_predictions} --task_name concept --experiment_config /home/david/bachelor-thesis/code/{experiment} {extra}" classification_job.condor'

outs = []
for experiment in sorted(all_experiments):
    name, name_ = experiment.split('/')[2:]
    assert name in priorities, 'Missing priority for experiment: {}'.format(name)
    prio = priorities.get(name)
    cmd = tmpl.format(name=name_, cores=cores, verbose=verbose, experiment=experiment, prio=prio, extra=extra, create_predictions=create_predictions)
    outs.append((prio, cmd))

cmd = 'condor_submit priority="{prio}" batch_name="{name}" Args="--n_jobs {cores} --verbose {verbose} --task_name concept --create_predictions {create_predictions}" classification_job.condor'.format(prio=-2, name='normal_concept_maps', cores=cores, verbose=verbose, create_predictions=create_predictions)
outs.append((-2, cmd))

cmd = 'condor_submit priority="{prio}" batch_name="{name}" Args="--n_jobs {cores} --verbose {verbose} --task_type_include text --create_predictions {create_predictions}" classification_job.condor'.format(prio=-1, name='text', cores=cores, verbose=verbose, create_predictions=create_predictions)
outs.append((-1, cmd))

for t in ['dummy_most_frequent', 'dummy_stratified', 'dummy_uniform']:
    cmd = ('condor_submit priority="{prio}" batch_name="{name}" Args="--n_jobs {cores} --verbose {verbose} --task_type_include {task_type}" classification_job.condor'.format(prio=10, name=t, task_type=t, cores=cores, verbose=verbose))
    outs.append((10, cmd))

cmds = ';\n\n'.join([cmd for prio, cmd in sorted(outs, key=lambda x: x[0], reverse=True)])
with open('tmp/start_classifaction_jobs.sh', 'w') as f:
    f.write('# Created: {}\n\n{}\n\n'.format(time_utils.get_time_formatted(), cmds))

print('# Jobs: {}'.format(len(outs)))

print('Uploading')
!chmod +x tmp/start_classifaction_jobs.sh
!scp tmp/start_classifaction_jobs.sh pe:condor_scripts/
print('Finished')